In [115]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from sklearn.metrics import plot_confusion_matrix, ConfusionMatrixDisplay, confusion_matrix

In [116]:
data_dir = (r'D:\Rostock-Masters\Thesis\Test')
data_dir

'D:\\Rostock-Masters\\Thesis\\Test'

In [117]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

WindowsPath('D:/Rostock-Masters/Thesis/Test')

In [118]:
file_name_dict = {
    'Device_1' : list(data_dir.glob('Device_1/*.csv')),
    'Device_2' : list(data_dir.glob('Device_2/*.csv')),
    'Device_3' : list(data_dir.glob('Device_3/*.csv')),
    'Device_4' : list(data_dir.glob('Device_4/*.csv')),
    'Device_5' : list(data_dir.glob('Device_5/*.csv')),
    'Device_6' : list(data_dir.glob('Device_6/*.csv'))
}

In [119]:
file_label_dict = {
    'Device_1' : 0,
    'Device_2' : 1,
    'Device_3' : 2,
    'Device_4' : 3,
    'Device_5' : 4,
    'Device_6' : 5,
}

In [120]:
len(file_name_dict['Device_4'])

100

In [121]:
str(file_name_dict['Device_3'][0])

'D:\\Rostock-Masters\\Thesis\\Test\\device_3\\3_8_5_3_8_5-1.csv'

### Classification

In [122]:
file_name_dict_2 = {
    'Device_1' : list(data_dir.glob('Device_1/*.csv')),
    'Device_2' : list(data_dir.glob('Device_2/*.csv'))
}

In [123]:
file_label_dict_2 = {
    'Device_1' : 0,
    'Device_2' : 1
}

In [124]:
df= pd.read_csv(r'C:\Users\mail2\Thesis\CNN\3_8_1_3_8_1-1.csv')
df.columns = ['time', 'voltage']
df.head()

,time,voltage
0,0.0,32.965541
1,0.5,32.914965
2,1.0,32.904850
3,1.5,32.864389
4,2.0,32.854273


In [125]:
df['MA'] = df['voltage'].rolling(10,min_periods=0).mean()
df.head()

,time,voltage,MA
0,0.0,32.965541,32.965541
1,0.5,32.914965,32.940253
2,1.0,32.904850,32.928452
3,1.5,32.864389,32.912436
4,2.0,32.854273,32.900804


### Preprocessing

In [126]:
device_imb_lst= []
device_list, device_label = [],[]   #compressed

for device_name, folder in file_name_dict.items():
    for file in folder:
        file_df = pd.read_csv(str(file))
        file_df.columns = ['time', 'voltage']
        file_df['MA'] = file_df['voltage'].rolling(10,min_periods=0).mean()# moving average
        file_df= file_df.drop('voltage',axis=1)
        file_df.rename(columns={'MA':'voltage'},inplace=True)

        threshold = file_df['voltage'].diff().gt(1)
        group = (threshold&~threshold.shift(fill_value=False)).cumsum().add(1)
        time= lambda i: i['time'].groupby(group).apply(lambda j: j- j.iloc[0])
        df_2 = (file_df.assign(bit=group,time=time).pivot(index='bit', columns='time', values='voltage'))

        df_3 = df_2.copy()
        df_3.reset_index(drop=True, inplace=True)
        df_3 = df_3.rename_axis(None, axis=1)

        #Compressing to  14 rows 100 columns
        df_4=df_3.iloc[:10, :120] #iloc = location, loc=label # 10 rows, 120 columns
        device_imb_lst.append(df_4)
        
        df_5= df_4.stack().reset_index(drop= True)
        df_6 = pd.DataFrame(df_5.values.reshape(1,-1))
        df_7 = df_6.iloc[:, :900]
        device_list.append(df_7)
        device_label.append(file_label_dict[device_name])
    
        
 #         df_4= df_4.fillna(method='pad')   

In [127]:
# # #Saving files to folder
# for i in range (len(device_list)):
#         device_list[i].to_csv('after_MA/file-{}.csv'.format(i))

In [200]:
device_imb_lst[350]

,0.0,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,...,55.0,55.5,56.0,56.5,57.0,57.5,58.0,58.5,59.0,59.5
0,31.084108,31.073993,31.090851,31.089166,31.078039,31.073993,31.068213,31.060084,31.057134,31.042635,...,29.812623,29.794416,29.792393,29.782278,29.776208,29.767105,29.762047,29.759013,29.738782,29.742828
1,28.499666,29.548616,30.503494,31.048705,31.573685,31.938845,32.235221,32.480010,32.662084,32.860343,...,30.375030,30.358846,30.345696,30.331535,30.312316,30.299166,30.283993,30.272866,30.254659,30.241509
2,26.444251,27.403175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,28.435940,29.487924,30.468090,31.024428,31.563570,31.928730,32.240279,32.478998,32.601393,32.823928,...,30.391215,30.370984,30.354800,30.336592,30.321419,30.304224,30.288039,30.270843,30.254659,30.240498
4,25.675493,26.705224,27.811830,28.925517,30.079665,31.150868,31.770932,32.362673,32.759190,33.109177,...,30.906080,30.882815,30.858538,30.836285,30.817066,30.799870,30.777616,30.759409,30.744236,30.725017
5,25.663355,26.708259,27.800704,28.969013,30.155530,31.271240,31.909511,32.521482,32.915976,33.281136,...,31.038589,31.015324,30.997117,30.975875,30.956656,30.938449,30.918218,30.895965,30.871688,30.858538
6,26.783111,27.919052,29.074211,30.269832,31.392622,32.034940,32.643877,33.050509,33.409600,33.705976,...,31.070958,31.044659,31.023417,31.007232,30.981944,30.966771,30.941483,30.924287,30.904057,30.884838
7,26.378502,27.360691,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,28.390422,29.436337,30.437745,30.996105,31.548397,31.907488,32.224095,32.501252,32.637808,32.834043,...,30.378065,30.356823,30.335581,30.315350,30.299166,30.282982,30.266797,30.249601,30.232405,30.216221
9,25.571306,26.588899,27.703597,28.803123,29.963340,31.062866,31.694056,32.293890,32.690407,33.039382,...,30.887872,30.863596,30.847411,30.822123,30.802904,30.777616,30.756374,30.741202,30.716925,30.701752


In [201]:
device_list[350]

,0,1,2,3,4,5,6,7,8,9,...,890,891,892,893,894,895,896,897,898,899
0,31.084108,31.073993,31.090851,31.089166,31.078039,31.073993,31.068213,31.060084,31.057134,31.042635,...,33.31654,33.258883,33.208307,33.154696,33.098051,33.042417,32.994875,32.93823,32.892711,32.841124


In [130]:
df_merged= pd.concat(device_list)
df_merged= df_merged.reset_index(drop=True)
df_merged

,0,1,2,3,4,5,6,7,8,9,...,890,891,892,893,894,895,896,897,898,899
0,32.965541,32.940253,32.928452,32.912436,32.900804,32.877876,32.858609,32.832779,32.804821,32.775375,...,33.381277,33.324632,33.270010,33.215387,33.159754,33.109177,33.056578,33.004990,32.952391,32.902827
1,30.679499,30.664326,30.655896,30.651682,30.651176,30.640724,30.637593,30.622600,30.619931,30.604646,...,33.392404,33.334747,33.280125,33.228537,33.169869,33.123339,33.064670,33.016117,32.957449,32.912942
2,32.743006,32.732891,32.712660,32.694959,32.680292,32.662084,32.643299,32.620359,32.596897,32.570035,...,33.377231,33.317551,33.267986,33.203249,33.147615,33.101085,33.043428,32.990829,32.938230,32.889677
3,31.306643,31.296528,31.299900,31.296528,31.288436,31.288098,31.274852,31.268711,31.249323,31.241906,...,33.416680,33.364081,33.305413,33.251802,33.193134,33.138512,33.081866,33.030279,32.971610,32.924069
4,31.660676,31.660676,31.650561,31.640446,31.624261,31.608414,31.592760,31.575961,31.558400,31.538282,...,33.387346,33.332724,33.278102,33.222468,33.167846,33.115246,33.063659,33.011060,32.960483,32.909907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,30.740190,30.654211,30.669383,30.633980,30.643084,30.630608,30.621697,30.613750,30.598577,30.588462,...,32.924069,32.874504,32.824939,32.779421,32.730868,32.682315,32.634773,32.583185,32.529575,32.484056
596,30.851458,30.836285,30.837971,30.836285,30.833250,30.834599,30.826892,30.816054,30.807625,30.795824,...,33.189088,33.139523,33.089958,33.041405,32.994875,32.950368,32.906873,32.856297,32.810778,32.756156
597,30.993071,30.998128,30.982956,30.988013,30.974863,30.974526,30.965615,30.958932,30.945866,30.936426,...,33.271021,33.211341,33.158742,33.104120,33.053544,33.001956,32.952391,32.903838,32.856297,32.804709
598,33.451072,33.430842,33.417355,33.403025,33.386335,33.366779,33.344140,33.323367,33.295972,33.265963,...,32.922046,32.870458,32.819882,32.771329,32.721764,32.665119,32.620612,32.569024,32.526540,32.482033


In [131]:
#Device label at the last column
df_with_label= df_merged.copy()
df_with_label['label'] = device_label
# df_with_label.to_csv('merged_with_label_2.csv', index= False)
df_with_label

,0,1,2,3,4,5,6,7,8,9,...,891,892,893,894,895,896,897,898,899,label
0,32.965541,32.940253,32.928452,32.912436,32.900804,32.877876,32.858609,32.832779,32.804821,32.775375,...,33.324632,33.270010,33.215387,33.159754,33.109177,33.056578,33.004990,32.952391,32.902827,0
1,30.679499,30.664326,30.655896,30.651682,30.651176,30.640724,30.637593,30.622600,30.619931,30.604646,...,33.334747,33.280125,33.228537,33.169869,33.123339,33.064670,33.016117,32.957449,32.912942,0
2,32.743006,32.732891,32.712660,32.694959,32.680292,32.662084,32.643299,32.620359,32.596897,32.570035,...,33.317551,33.267986,33.203249,33.147615,33.101085,33.043428,32.990829,32.938230,32.889677,0
3,31.306643,31.296528,31.299900,31.296528,31.288436,31.288098,31.274852,31.268711,31.249323,31.241906,...,33.364081,33.305413,33.251802,33.193134,33.138512,33.081866,33.030279,32.971610,32.924069,0
4,31.660676,31.660676,31.650561,31.640446,31.624261,31.608414,31.592760,31.575961,31.558400,31.538282,...,33.332724,33.278102,33.222468,33.167846,33.115246,33.063659,33.011060,32.960483,32.909907,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,30.740190,30.654211,30.669383,30.633980,30.643084,30.630608,30.621697,30.613750,30.598577,30.588462,...,32.874504,32.824939,32.779421,32.730868,32.682315,32.634773,32.583185,32.529575,32.484056,5
596,30.851458,30.836285,30.837971,30.836285,30.833250,30.834599,30.826892,30.816054,30.807625,30.795824,...,33.139523,33.089958,33.041405,32.994875,32.950368,32.906873,32.856297,32.810778,32.756156,5
597,30.993071,30.998128,30.982956,30.988013,30.974863,30.974526,30.965615,30.958932,30.945866,30.936426,...,33.211341,33.158742,33.104120,33.053544,33.001956,32.952391,32.903838,32.856297,32.804709,5
598,33.451072,33.430842,33.417355,33.403025,33.386335,33.366779,33.344140,33.323367,33.295972,33.265963,...,32.870458,32.819882,32.771329,32.721764,32.665119,32.620612,32.569024,32.526540,32.482033,5


In [132]:
#Device label at the first column
df_with_label_2= df_merged
df_with_label_2.insert(0,'label',device_label)
# df_with_label_2.to_csv('merged_with_label.csv', index= False)

In [133]:
#Checking null values in entire dataframe
df_merged.isnull().values.any()

False

In [134]:
df_new= device_list[402]
df_new

,0,1,2,3,4,5,6,7,8,9,...,890,891,892,893,894,895,896,897,898,899
0,33.147615,33.112212,33.097039,33.079337,33.06467,33.044777,33.017562,32.994622,32.967789,32.943288,...,33.205272,33.149638,33.097039,33.043428,32.991841,32.939241,32.886642,32.835055,32.775375,32.727833


In [135]:
df_new_2= df_new.stack().reset_index(drop= True)
df_new_2

0      33.147615
1      33.112212
2      33.097039
3      33.079337
4      33.064670
         ...    
895    32.939241
896    32.886642
897    32.835055
898    32.775375
899    32.727833
Length: 900, dtype: float64

In [136]:
# df_new_2= df_new_2.transpose()
# df_new_2

In [137]:
pv_df = pd.DataFrame(df_new_2.values.reshape(1,-1))
# pv_df.to_csv('single_row.csv')